In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import seaborn as sns
import os
import warnings

# 파일 불러와서 열이름 바꾸기
korean, long_term_frgn, short_term_frgn, resident, card_cnt, card_amt, waste_cnt
(이거 예시는 card_amt임)

In [2]:
df = pd.read_csv("files\\data_preprocessing\\long_term_frgn_preprocessing.csv", encoding='cp949')

# 시계열 피처단위로 변경
df['base_date'] = pd.to_datetime(df['base_date'])

# base_date를 index로 설정
df.index = df['base_date']
df.columns = ['base_date', 'emd_cd', 'long_term_frgn']  
df

,base_date,emd_cd,long_term_frgn
base_date,,,
2018-01-31,2018-01-31,50110250,427986.3363
2018-02-28,2018-02-28,50110250,427240.8119
2018-03-31,2018-03-31,50110250,513638.2631
2018-04-30,2018-04-30,50110250,474601.3510
2018-05-31,2018-05-31,50110250,395262.5483
...,...,...,...
2021-02-28,2021-02-28,50130620,109325.4866
2021-03-31,2021-03-31,50130620,118258.8891
2021-04-30,2021-04-30,50130620,129227.7480


In [3]:
# 필요 없는 열 제거
data = df.drop(['base_date'], 1)    
data

,emd_cd,long_term_frgn
base_date,,
2018-01-31,50110250,427986.3363
2018-02-28,50110250,427240.8119
2018-03-31,50110250,513638.2631
2018-04-30,50110250,474601.3510
2018-05-31,50110250,395262.5483
...,...,...
2021-02-28,50130620,109325.4866
2021-03-31,50130620,118258.8891
2021-04-30,50130620,129227.7480


In [4]:
# 한 행정동의 행만 추출
sub_area = data['emd_cd'] == 50110640

# base_date, card_amt 만 갖는 df 생성
sub_df = data[sub_area]
sub_df = sub_df.drop('emd_cd', 1)
sub_df

,long_term_frgn
base_date,
2018-01-31,156502.4655
2018-02-28,123070.7676
2018-03-31,128927.3110
2018-04-30,121078.3448
2018-05-31,119408.6076
2018-06-30,122454.5316
2018-07-31,166255.4576
2018-08-31,156265.6981
2018-09-30,145615.9811


In [5]:
# series 형으로 변환
sub_df.to_dict('series')

{'long_term_frgn': base_date
 2018-01-31    156502.4655
 2018-02-28    123070.7676
 2018-03-31    128927.3110
 2018-04-30    121078.3448
 2018-05-31    119408.6076
 2018-06-30    122454.5316
 2018-07-31    166255.4576
 2018-08-31    156265.6981
 2018-09-30    145615.9811
 2018-10-31    130634.8717
 2018-11-30    138396.0819
 2018-12-31    159311.9488
 2019-01-31    211525.4804
 2019-02-28    159330.8430
 2019-03-31    207181.4331
 2019-04-30    161679.3823
 2019-05-31    168119.4102
 2019-06-30    164454.8750
 2019-07-31    195203.8806
 2019-08-31    190007.4237
 2019-09-30    149453.6223
 2019-10-31    168765.4625
 2019-11-30    158626.1745
 2019-12-31    190723.0042
 2020-01-31    159049.0104
 2020-02-29    143706.6857
 2020-03-31    129295.2232
 2020-04-30    141012.1307
 2020-05-31    126854.1440
 2020-06-30    131770.2199
 2020-07-31    143850.7588
 2020-08-31    127930.3162
 2020-09-30    118117.1436
 2020-10-31    142042.9204
 2020-11-30    123298.0098
 2020-12-31    133255.1862

# ARIMA

In [6]:
# ARIMA에 적합한 float로 바꾸는 과정
sub_df_float = sub_df[:].astype(np.float)

In [7]:
# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
from statsmodels.tsa.stattools import adfuller

result = adfuller(sub_df)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -2.047732
p-value : 0.266044
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [8]:
# 차분
diff_1 = sub_df_float.diff(periods=1).iloc[1:]

# ADF 검정 - 정상성 확인하는 검정 (귀무 : 정상성 만족 x, 대립 : 정상성 만족)
result = adfuller(diff_1)
print('ADF Statistic : %f' % result[0])
print('p-value : %f' % result[1])
print('Critical Values :')
for key, value in result[4].items():
    print('\t%s: %3f' % (key, value))

ADF Statistic : -10.302983
p-value : 0.000000
Critical Values :
	1%: -3.605565
	5%: -2.937069
	10%: -2.606986


In [9]:
from pmdarima.arima import auto_arima

model_arima= auto_arima(sub_df,trace=True, error_action='ignore',suppress_warnings=True,stepwise=False,seasonal=True)

model_arima.fit(sub_df)

 ARIMA(0,0,0)(0,0,0)[1] intercept   : AIC=972.550, Time=0.01 sec
 ARIMA(0,0,1)(0,0,0)[1] intercept   : AIC=966.918, Time=0.09 sec
 ARIMA(0,0,2)(0,0,0)[1] intercept   : AIC=962.094, Time=0.05 sec
 ARIMA(0,0,3)(0,0,0)[1] intercept   : AIC=963.289, Time=0.06 sec
 ARIMA(0,0,4)(0,0,0)[1] intercept   : AIC=963.580, Time=0.11 sec
 ARIMA(0,0,5)(0,0,0)[1] intercept   : AIC=961.730, Time=0.21 sec
 ARIMA(1,0,0)(0,0,0)[1] intercept   : AIC=958.755, Time=0.02 sec
 ARIMA(1,0,1)(0,0,0)[1] intercept   : AIC=958.284, Time=0.05 sec
 ARIMA(1,0,2)(0,0,0)[1] intercept   : AIC=959.543, Time=0.08 sec
 ARIMA(1,0,3)(0,0,0)[1] intercept   : AIC=960.259, Time=0.06 sec
 ARIMA(1,0,4)(0,0,0)[1] intercept   : AIC=965.980, Time=0.10 sec
 ARIMA(2,0,0)(0,0,0)[1] intercept   : AIC=956.610, Time=0.05 sec
 ARIMA(2,0,1)(0,0,0)[1] intercept   : AIC=958.406, Time=0.05 sec
 ARIMA(2,0,2)(0,0,0)[1] intercept   : AIC=959.899, Time=0.15 sec
 ARIMA(2,0,3)(0,0,0)[1] intercept   : AIC=961.237, Time=0.16 sec
 ARIMA(3,0,0)(0,0,0)[1] i

ARIMA(order=(2, 0, 0), scoring_args={}, seasonal_order=(0, 0, 0, 1),
      suppress_warnings=True)

In [10]:
# ARIMA 모델 
from statsmodels.tsa.arima_model import ARIMA
import statsmodels.api as sm

# (AR=2, 차분=1, MA=2) 파라미터로 ARIMA 모델을 학습합니다.
model = ARIMA(sub_df.long_term_frgn.values, order=(2,0,0))

#trend : constant를 가지고 있는지, c - constant / nc - no constant
#disp : 수렴 정보를 나타냄
model_fit = model.fit(trend='c', full_output=True, disp=True)
print(model_fit.summary())

                              ARMA Model Results                              
Dep. Variable:                      y   No. Observations:                   42
Model:                     ARMA(2, 0)   Log Likelihood                -474.291
Method:                       css-mle   S.D. of innovations          19286.947
Date:                Sat, 11 Sep 2021   AIC                            956.581
Time:                        04:16:46   BIC                            963.532
Sample:                             0   HQIC                           959.129
                                                                              
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const       1.471e+05   8755.628     16.806      0.000     1.3e+05    1.64e+05
ar.L1.y        0.3714      0.146      2.541      0.011       0.085       0.658
ar.L2.y        0.3112      0.148      2.100      0.0

C:\Users\soeun\anaconda3\lib\site-packages\statsmodels\tsa\arima_model.py:472: FutureWarning: 
statsmodels.tsa.arima_model.ARMA and statsmodels.tsa.arima_model.ARIMA have
been deprecated in favor of statsmodels.tsa.arima.model.ARIMA (note the .
between arima and model) and
statsmodels.tsa.SARIMAX. These will be removed after the 0.12 release.

statsmodels.tsa.arima.model.ARIMA makes use of the statespace framework and
is both well tested and maintained.

To silence this warning and continue using ARMA and ARIMA until they are
removed, use:

import warnings
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARMA',
                        FutureWarning)
warnings.filterwarnings('ignore', 'statsmodels.tsa.arima_model.ARIMA',
                        FutureWarning)

  warnings.warn(ARIMA_DEPRECATION_WARN, FutureWarning)


# 예측하기

In [11]:
# 2단위 이후의 예측결과
fore = model_fit.forecast(steps=2)
print(fore)

(array([140906.88339617, 144717.34258463]), array([19286.94672909, 20574.03360734]), array([[103105.16243541, 178708.60435693],
       [104392.97769754, 185041.70747173]]))
